https://drive.google.com/open?id=1OletxmPYNkz2ltOr9pyT0b0iBtUWxslh)
Here it is of the format:
word \t label\n
for instance:
postural	B
hypotension	I

here B-> Begin entity, I-> inside entity and O-> outside entity



In [3]:
#Import all required libraries
import nltk
from nltk.corpus import stopwords
import spacy
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lodha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lodha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
sent=input("Enter the sentence")
tok_sent=nltk.word_tokenize(sent)
print(tok_sent)

['Ayush', 'is', 'suffering', 'from', 'cold', 'and', 'fever']


In [5]:
sent2=[]
act_sent=[]
stop_words=stopwords.words('english')
for word in tok_sent:
    sent2.append(word.lower())
print(sent2)
for i in sent2:
    if (i not in stop_words):
        act_sent.append(i)
print(act_sent)

['ayush', 'is', 'suffering', 'from', 'cold', 'and', 'fever']
['ayush', 'suffering', 'cold', 'fever']


In [8]:
fin_sent=""
for i in act_sent:
    fin_sent=fin_sent+" "+i
print(fin_sent)

 ayush suffering cold fever


We have to convert tsv file to the format accepted by spaCy for training.
One of the format supported by spaCy is:
TRAIN_DATA = [[(Sentence, {entities: [(start, end, label)]], ...]

In [11]:
def load_data_spacy(file_path):
    ''' Converts data from:
    word \t label \n word \t label \n \n word \t label
    to: sentence, {entities : [(start, end, label), (stard, end, label)]}
    '''
    file = open(file_path, 'r')
    training_data, entities, sentence, unique_labels = [], [], [], []
    current_annotation = None
    start =0
    end = 0 # initialize counter to keep track of start and end characters
    for line in file:
        line = line.strip("\n").split("\t")
        # lines with len > 1 are words
        if len(line) > 1:
            label = line[1]
            if(label != 'O'):
                label = line[1]+"_Disease"     # the .txt is formatted: label \t word, label[0:2] = label_type
            #label_type = line[0][0] # beginning of annotations - "B", intermediate - "I"
            word = line[0]
            sentence.append(word)
            start = end
            end += (len(word) + 1)  # length of the word + trailing space
           
            if label == 'I_Disease' :  # if at the end of an annotation
                entities.append(( start,end-1, label))  # append the annotation
                              
            if label == 'B_Disease':                         # if beginning new annotation
                entities.append(( start,end-1, label))# start annotation at beginning of word
                
           
           
            if label != 'O' and label not in unique_labels:
                unique_labels.append(label)
 
        # lines with len == 1 are breaks between sentences
        if len(line) == 1:
            if(len(entities) > 0):
                sentence = " ".join(sentence)
                training_data.append([sentence, {'entities' : entities}])
            # reset the counters and temporary lists
            end = 0 
            start = 0
            entities, sentence = [], []
            
    file.close()
    return training_data, unique_labels   

Let us convert our train data,test data and validation data to spaCy format

In [ ]:
TRAIN_DATA, LABELS = load_data_spacy(r"C:\Users\lodha\Downloads\CustomeNERspaCyv3.0-main (2)\CustomeNERspaCyv3.0-main\train.tsv")
print(TRAIN_DATA)
print(len(TRAIN_DATA))
TEST_DATA, _ = load_data_spacy(r"C:\Users\lodha\Downloads\CustomeNERspaCyv3.0-main (2)\CustomeNERspaCyv3.0-main\test.tsv")
print(len(TEST_DATA))
VALID_DATA, _ = load_data_spacy(r"C:\Users\lodha\Downloads\CustomeNERspaCyv3.0-main (2)\CustomeNERspaCyv3.0-main\train_dev.tsv")
print(len(VALID_DATA))


https://stackoverflow.com/questions/67407433/using-spacy-3-0-to-convert-data-from-old-spacy-v2-format-to-the-brand-new-spacy  The below method is used to convert the train and validation data from old format to new format

In [20]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

db = DocBin()
for text, annot in tqdm(VALID_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./valid.spacy") # save the docbin object

100%|█████████████████████████████████████████████████████████████████████████████| 5385/5385 [00:09<00:00, 582.79it/s]


In [7]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values

2022-03-16 10:43:35.114409: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-16 10:43:35.114493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Train using CLI

In [ ]:
 !python -m spacy train config.cfg --verbose --output ./ner_demo/training/ --paths.train train.spacy --paths.dev valid.spacy

Lets test our model on  test data

In [ ]:
ner = spacy.load(R"ner_demo/training/model-best") #load the best model



test_sentences = [x[0] for x in TEST_DATA[0:4000]] # extract the sentences from [sentence, entity]
for x in test_sentences:
    doc = ner(x)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
    displacy.render(doc,jupyter=True, style = "ent")
   

In [12]:
import spacy
#fin_sent = "ayush suffering cold fever"
ner = spacy.load(R"ner_demo/training/model-best") #load the best model
doc = ner(fin_sent)
result = ""
for ent in doc.ents:
    texts = ent.text
    label = ent.label_
    result = result + texts + " : " + label + "\n"
print(result)
displacy.render(doc,jupyter=True, style = "ent")

cold : B_Disease
fever : I_Disease

